# Simple Filtering

In this section we will discuss filtering data. Filtering data is important, as All of Us will not always do this for you initially, and often in analysis you need to filter by subsets anyway. Filtering can get complex, so this section is split into two notebooks. This one covers simple filters, while the next one covers creating new data labels that one can then filter on.

### Filtering Using Mathematical Operators
To get started with filtering we need a dataset. The code below will make a dataframe for us. In fact, it will be the same dataframe as last notebook, but without random blank cells.

In [ ]:
import pandas as pd
import random

workout_dict = {'ID': [], 'Measurement Device': [], 'Heart Rate Max': [], 'Heart Rate Min': [], 'Heart Rate Avg': [],
              'Duration of exercise (min)': [], 'Exercise Type': []}
used_ids = []

for x in range(0, 500):
    id = random.randint(100000000, 999999999)
    while id in used_ids:
        id = random.randint(100000000, 999999999)
    used_ids.append(id)
    device = random.choice(['Skykandal', 'B-Wolf'])
    mu = random.randint(65, 85)
    min_rate = int(random.gauss(mu, 10))
    max_rate = int(random.gauss(mu + 55, 25))
    while max_rate <= min_rate:
        max_rate = int(random.gauss(mu + 55, 25))
    avg = random.gauss((max_rate + min_rate) / 2, (max_rate - min_rate) / 5)
    duration = random.randint(10, 90)
    exercise = random.choice(['Running', 'Running', 'Running', 'Bicycling', 'Swimming', 'Swimming',
                              'Weight training'])
    row = [device, min_rate, max_rate, avg, duration, exercise]
    workout_dict['ID'].append(id)
    workout_dict['Measurement Device'].append(row[0])
    workout_dict['Heart Rate Min'].append(row[1])
    workout_dict['Heart Rate Max'].append(row[2])
    workout_dict['Heart Rate Avg'].append(row[3])
    workout_dict['Duration of exercise (min)'].append(row[4])
    workout_dict['Exercise Type'].append(row[5])

df = pd.DataFrame(workout_dict)
df.head(10)

Let's start with a simple filter: we'll filter out everyone whose minimum heart rate is under 70. (We could also say that we'll filter for everyone whose heart rate is above 69.)

In [ ]:
df[df['Heart Rate Min'] > 69]

How does this work? Well, we start with `df[]` which is a common way to get something out of dataframe `df` using keywords. Except, instead of passing something like a column name, we passed an expression into the brackets. The expression was `df['Heart Rate Min'] > 69'`. In other words, from df give me every row where Heart Rate Min (in df) is greater than 69.

Now, let's get one of the most annoying features of pandas out of the way. We're going to store our filtered data in df_filtered and then, for no good reason, change a column.

In [ ]:
df_filtered = df[df['Heart Rate Min'] > 69]
df_filtered['Heart Rate Avg'] = 200
df_filtered.head()

Above you should have gotten a `SettingWithCopyWarning`. You might have assumed that when you assigned `df[df['Age'] > 29]` to `df_filtered` you created an entirely new dataframe, named df_filtered. You didn't. You created a "view" of the old dataframe (df) that only saw the parts of df that you wanted, but wasn't actually a new dataframe. This means that attempting to set a value in df_filtered may also alter values in df. And so we get a warning.

When you get a view and when you get a new pandas object is fairly complicated. However, there is a simple fix: `copy` can turn a view into a copy. Copy is a dataframe method, and takes no arguments.

Below, the code has added `copy()` and what previously gave us a warning now won't.

In [ ]:
df_filtered = df[df['Heart Rate Min'] > 69].copy()
df_filtered['Heart Rate Avg'] = 200
df_filtered.head()

Filtering expressions recognize a number of comparison operators: >, <, <=, >=, ==, and !=. Remember that in Python = means "set the thing on the left side of the = equal to the thing on the right side of the =", whereas == asks if they are equal. != means "not equal".

These are fairly straightforward when we are dealing with numbers. 
#### Below, write some code that filters df so that only individuals with a maximum heart rate of 110 or under remain.

In [ ]:
# your code goes here


You can also use some of these, like == and != to filter on text columns. Below, we'll just select the individuals who went running.

In [ ]:
df[df['Exercise Type'] == 'Running']

We could also exclude people who went running using ~. This means "not", and it's generally good practice (i.e., it will sometimes break if you don't do this) to wrap the expression you are negating in parentheses, as below.

In [ ]:
df[~(df['Exercise Type'] == 'Running')]

What happens if we use a > or < symbol?

In [ ]:
df[df['Exercise Type'] < 'Running']

`< 'Running'` gives us only bicyclists, because > and < on text check alphabetization. < 'Running' gave us every label that is, when alphabetized, earlier than "Running". This behavior is rarely useful, and this is mostly a warning that these operators will do something if used, but probably not what you want.

#### Below, write code that returns only people using the B-Wolf measuring device. Do this two different ways. To make it easier to see that each way worked there are separate cells for each way.

In [ ]:
# your code goes here for the first way


In [ ]:
# the second way goes here


What if we wanted to filter based on the relationship between two columns? We can pass more complex expressions to the filter as well. The filter below will only return individuals whose maximum heart rate is within 20 beats of their minimum heart rate.

In [ ]:
df[(df['Heart Rate Max'] - df['Heart Rate Min']) <= 20]

#### Below, write code that returns everyone whose average heart rate was less than the number you would get by averaging the maximum and minimum heart rates. It's easy to miss parentheses and brackets when writing something this long, so I suggest that you write in your parentheses/brackets first, and then fill them in.

In [ ]:
# your code goes here


### Filtering Using isin()
The filtering tools above work fine for text columns where we want only one value, but what if we want more than one value? The `isin` method is our answer.

In [ ]:
df[df['Exercise Type'].isin(['Running', 'Swimming'])]

The `isin` method is very similar to the basic Python `in`. It simply checks to see if something is in an iterable that you provide. In this case we simply passed the list `['Running', 'Swimming']` to `isin` and got back every row where the value in the Condition column was in the list.

If we had a lot of categories and we wanted to get most, but not all, of them we could in theory use `isin` and write a long list of categories to be included. However, it would be nicer to exclude the small list. Thankfully, ~ works for `isin` just like it did for mathematical expressions

See below for usage.

In [ ]:
df[~df['Exercise Type'].isin(['Running', 'Swimming'])]

While you might be tempted to try and write `df.~isin` Python interprets that as nonsense. In reality. `df.isin` is creating a dataframe full of `True` and `False`, and we're asking for that frame to be flipped, so every `True` is a `False` and vice-versa.

#### For a bit of quick practice, make a filter that returns everyone who wasn't bicycling or running.

In [ ]:
# your code goes here


### Chaining Filters
Now, we could combine filters by chaining filters together in the following manner:

In [ ]:
temp_df = df[df['Heart Rate Min'] > 80]
temp_df[temp_df['Measurement Device'] == 'Skykandal']

However, this is ugly and requires the creation of potentially a large number of temporary dataframes that exist only to hold intermediate steps. We could do the same thing a different way: write each expression in parentheses and then connect them together.

Below is the same result as above but in one line.

In [ ]:
df[(df['Heart Rate Min'] > 80) & (df['Measurement Device'] == 'Skykandal')]

All we've done here is connect two expressions we already understand with &. Python evaluates each expression (`df['Heart Rate Min'] > 80` and `df['Measurement Device'] == 'Skykandal'`) separately. If they both evaluate to True then the whole statement evaluates to True.

#### Write some code below that filters for all people who were swimming and spent more than 60 minutes doing so.

In [ ]:
# your code goes here


There are other ways to connect expressions. & handles "and", meaning both parts have to be true. | (which you can generally type as Shift+\) means "or", meaning at least one of the parts has to be true.

We could express this like so:
True & True = True
False & True = False
False | True = True
True | True = True

Between &, |, and ~ you can build some fairly complex expressions, especially since you can connect blocks inside parentheses with other blocks.

The code block below filters for everyone with an average heart rate of 120-130 or 140-150. Visually, the line is broken at the | so you can more easily see the two halves.

In [ ]:
df[((df['Heart Rate Avg']>= 120) & (df['Heart Rate Avg'] <= 130)) | 
   ((df['Heart Rate Avg'] >= 140) & (df['Heart Rate Avg'] <= 150))]

There's more than one way to do this. The code block below will give the same results as the one above. Spend a minute looking at it until you can tell why. It may help to think about both filters in chunks in pieces (both filters are two sets of two linked expressions).

In [ ]:
df[~((df['Heart Rate Avg'] < 120) | (df['Heart Rate Avg'] > 150)) &
   ~((df['Heart Rate Avg'] > 130) & (df['Heart Rate Avg'] < 140))]

Remember how I said that making intermediate data frames is bad practice? When the expression gets complicated enough you sometimes need to do so for debugging purposes. However, you can potentially build extremely powerful filters this way.

#### Below, write code that filters for people who went running or swimming while using the B-Wolf device OR who went bicycling using Starkandal, and have an average heart rate below the number you get by averaging their maximum and minimum heart rates together.

In [ ]:
# your code goes here


At this point you may be thinking, "Wow, I wish I could write a function, pass all of these rows to that function, evaluate the row there, and then just return a True/False label I could filter against."

That's a great idea! We'll cover it in the next notebook.